<a href="https://colab.research.google.com/github/Vince2kLyleStyle/lylepinkard/blob/main/Leverage_Bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q pandas requests gspread oauth2client

In [2]:
from google.colab import files
import shutil

print("Upload your Google service account JSON")
uploaded = files.upload()
name = next(iter(uploaded))
shutil.move(f"/content/{name}", "/content/service_account.json")
print("Saved to /content/service_account.json")


Upload your Google service account JSON


Saving ecstatic-device-470522-i7-5d3eebbfd65a.json to ecstatic-device-470522-i7-5d3eebbfd65a.json
Saved to /content/service_account.json


In [ ]:
# ...existing code...
import time
import traceback
import requests
import pandas as pd
import threading
import json
from datetime import datetime, timezone
from collections import deque, defaultdict
from typing import Optional

# Optional websocket-client
try:
    import websocket  # pip install websocket-client
    WEBSOCKET_AVAILABLE = True
except Exception:
    websocket = None
    WEBSOCKET_AVAILABLE = False

# ================== CONFIG ==================
TELEGRAM_TOKEN = "7555574651:AAFYxc3mztygEsX6iaUUU4DJSFm4GV82KZA"
TELEGRAM_CHAT_ID = "6218092459"
DISCORD_WEBHOOK_ALL = "https://discord.com/api/webhooks/1417909920690344127/n1nUIz757r7mlgJrV-vDKzRbU03MOfsLHYYmEGWNR2iuDezigHeVabsMCUgrsi-4cVHy"
DISCORD_WEBHOOK_PRIORITY = "https://discord.com/api/webhooks/1420867254496657469/3vS4BNJ8m9wx05Zx1XAcfFU3jHVXYV3uGaGRVAv4xXaUnewZrViGBST-8g2Q50KpmGRF"
DISCORD_PRIORITY_ROLE_ID = "1418151507466915930"

PER_TRADE_RISK_USD = 10
TELEGRAM_CONF_THRESHOLD = 65
PRIORITY_CONF_THRESHOLD = 75

MARKETS = ["BTC", "ETH", "SOL", "XRP", "AVAX"]
COINBASE_SYMBOLS = {
    "BTC": "BTC-USD",
    "ETH": "ETH-USD",
    "SOL": "SOL-USD",
    "XRP": "XRP-USD",
    "AVAX": "AVAX-USD",
}

# Spam control / throttles
PRIO_MIN_GAP = 90
REG_MIN_GAP = 45
MAN_MIN_GAP = 90
PRIO_BURST_WINDOW_SEC = 600
PRIO_BURST_COUNT = 3

# Polling and live thresholds
POLL_INTERVAL = 30
LIVE_DEVIATION_BP = 0.0012  # 12bp

# Google Sheets (optional) - set to "" to disable
GSHEET_JSON = "ecstatic-device-470522-i7-5d3eebbfd65a.json"
GSHEET_NAME = "Paper Trades"
sheet = None

# Debug
DEBUG_COMPONENTS = True

# ================== STATE ==================
LAST_SENT = defaultdict(lambda: {"prio_ts": 0, "reg_ts": 0, "man_ts": 0, "fp": ""})
PRIO_BURST = defaultdict(lambda: deque(maxlen=5))
CONF_HISTORY = defaultdict(lambda: deque(maxlen=120))
LAST_PRIORITY_TS = defaultdict(lambda: 0)
DROUGHT_PROMOTED_COUNT = defaultdict(lambda: 0)
LAST_CANDLE_TS = {}
LIVE_LAST = {}

# ================== HELPERS ==================
def now_ts() -> int:
    return int(time.time())

def utcnow() -> datetime:
    return datetime.now(timezone.utc)

def init_sheet():
    global sheet
    if not GSHEET_JSON:
        sheet = None
        return
    try:
        import gspread
        from oauth2client.service_account import ServiceAccountCredentials
        scope = ["https://spreadsheets.google.com/feeds","https://www.googleapis.com/auth/drive"]
        creds = ServiceAccountCredentials.from_json_keyfile_name(GSHEET_JSON, scope)
        gc = gspread.authorize(creds)
        sheet = gc.open(GSHEET_NAME).sheet1
        print("[INFO] Google Sheets initialized")
    except Exception as e:
        print("[WARN] Sheets init failed:", e)
        sheet = None

def log_sheet(row):
    if not sheet:
        return
    try:
        sheet.append_row(row)
    except Exception as e:
        print("[WARN] Sheet append failed:", e)

# ================== DATA FETCH ==================
def fetch_coinbase_candles(symbol: str, interval: str, lookback_minutes: int) -> pd.DataFrame:
    granularity = int(interval) * 60
    end = int(time.time() // granularity * granularity)
    start = end - lookback_minutes * 60
    start = int(start // granularity * granularity)
    start_dt = datetime.fromtimestamp(start, tz=timezone.utc)
    end_dt = datetime.fromtimestamp(end, tz=timezone.utc)
    params = {
        "start": start_dt.isoformat(timespec='seconds').replace('+00:00', 'Z'),
        "end": end_dt.isoformat(timespec='seconds').replace('+00:00', 'Z'),
        "granularity": granularity
    }
    try:
        if DEBUG_COMPONENTS:
            print(f"[INFO] Fetching {symbol} {interval}m candles...")
        r = requests.get(f"https://api.exchange.coinbase.com/products/{symbol}/candles", params=params, timeout=15)
        r.raise_for_status()
        data = r.json()
    except Exception as e:
        print(f"[WARN] Failed to fetch {symbol} {interval}m: {e}")
        return pd.DataFrame()
    if not data or not isinstance(data, list):
        return pd.DataFrame()
    df = pd.DataFrame(data, columns=['time','low','high','open','close','volume'])
    df['time'] = pd.to_datetime(df['time'], unit='s', utc=True)
    for c in ['open','high','low','close','volume']:
        df[c] = pd.to_numeric(df[c], errors='coerce')
    df = df.dropna().sort_values('time').reset_index(drop=True)
    return df

# ================== INDICATORS ==================
def add_indicators(df: pd.DataFrame) -> pd.DataFrame:
    if df is None or df.empty:
        return df
    df = df.copy()
    span20 = min(20, max(2, len(df)))
    span50 = min(50, max(2, len(df)))
    span200 = min(200, max(2, len(df)))
    df["EMA20"] = df["close"].ewm(span=span20, adjust=False).mean()
    df["EMA50"] = df["close"].ewm(span=span50, adjust=False).mean()
    df["EMA200"] = df["close"].ewm(span=span200, adjust=False).mean()
    prev = df["close"].shift(1)
    tr = pd.concat([
        df["high"] - df["low"],
        (df["high"] - prev).abs(),
        (df["low"] - prev).abs()
    ], axis=1).max(axis=1)
    df["ATR14"] = tr.rolling(window=min(14, max(1, len(df))), min_periods=1).mean()
    return df

def ema_side_slope(df: pd.DataFrame):
    if df is None or "EMA50" not in df or len(df) < 5:
        return "na", "na"
    side = "above" if df["close"].iloc[-1] > df["EMA50"].iloc[-1] else "below"
    slope = "up" if df["EMA50"].iloc[-1] > df["EMA50"].iloc[-5] else "down"
    return side, slope

def trend_score(df1h: pd.DataFrame, df5m: pd.DataFrame) -> int:
    s = 0
    for df in [df1h, df5m]:
        side, slope = ema_side_slope(df)
        if side == "above" and slope == "up":
            s += 1
        elif side == "below" and slope == "down":
            s -= 1
    return s

def compute_session_vwap(df1m: pd.DataFrame) -> Optional[float]:
    if df1m is None or df1m.empty:
        return None
    last_dt = df1m["time"].iloc[-1]
    if pd.isna(last_dt):
        return None
    session_date = pd.to_datetime(last_dt).date()
    mask = df1m["time"].dt.date == session_date
    if not mask.any():
        return None
    part = df1m.loc[mask]
    tp = (part["high"] + part["low"] + part["close"]) / 3.0
    vol = part["volume"].clip(lower=1e-12)
    vwap = (tp * vol).cumsum() / vol.cumsum()
    return float(vwap.iloc[-1]) if not vwap.empty else None

def five_min_side(df5m: pd.DataFrame) -> str:
    if df5m is None or df5m.empty or "EMA20" not in df5m or "EMA50" not in df5m:
        return "na"
    return "above" if df5m["EMA20"].iloc[-1] > df5m["EMA50"].iloc[-1] else "below"

def one_min_trigger(df1m: pd.DataFrame, side: str) -> bool:
    try:
        if df1m is None or df1m.empty:
            return True
        e20 = df1m["close"].ewm(span=min(20, max(1, len(df1m))), adjust=False).mean()
        c1 = df1m["close"].iloc[-1]
        c2 = df1m["close"].iloc[-2] if len(df1m) >= 2 else c1
        ema20_last = e20.iloc[-1]
        ema20_prev = e20.iloc[-2] if len(e20) >= 2 else ema20_last
        if side == "BUY":
            return (c1 > ema20_last and c2 > ema20_prev) or (ema20_last > ema20_prev)
        else:
            return (c1 < ema20_last and c2 < ema20_prev) or (ema20_last < ema20_prev)
    except Exception:
        return True

def ema_slope_bonus_5m(df5m: pd.DataFrame, side: str) -> int:
    try:
        if df5m is None or df5m.empty or "EMA50" not in df5m:
            return 0
        e = df5m["EMA50"].iloc[-5:]
        if len(e) < 5:
            return 0
        rising = e.iloc[-1] > e.iloc[0]
        return 3 if (rising and side == "BUY") or ((not rising) and side == "SELL") else 0
    except Exception:
        return 0

def atr_clamp_ok(stop_width: float, atr1m: Optional[float], lo_mult: float = 1.0, hi_mult: float = 2.5) -> bool:
    if atr1m is None or atr1m <= 0:
        return True
    return (lo_mult * atr1m) <= stop_width <= (hi_mult * atr1m)

def detect_swing(df: pd.DataFrame, side: str) -> Optional[float]:
    if df is None or df.empty or len(df) < 3:
        return None
    h = df["high"]
    l = df["low"]
    swing_highs = h[(h > h.shift(1)) & (h > h.shift(-1))].dropna()
    swing_lows = l[(l < l.shift(1)) & (l < l.shift(-1))].dropna()
    if side == "SELL" and not swing_highs.empty:
        return float(swing_highs.iloc[-1])
    if side == "BUY" and not swing_lows.empty:
        return float(swing_lows.iloc[-1])
    return None

# ================== SCORING & PRIORITY ==================
def compute_confidence(regime_score, ema5m_side, side, vwap_align, stop_width, atr1m) -> int:
    conf = 0
    if abs(regime_score) >= 2:
        conf += 30
    elif regime_score != 0:
        conf += 15
    if (ema5m_side == "below" and side == "SELL") or (ema5m_side == "above" and side == "BUY"):
        conf += 20
    if vwap_align:
        conf += 10
    if atr1m and stop_width:
        if 1.2 * atr1m <= stop_width <= 3.0 * atr1m:
            conf += 15
        elif 0.8 * atr1m <= stop_width <= 3.5 * atr1m:
            conf += 8
    return int(min(100, conf))

def is_priority(setup: dict, regime_score: float, conf: int) -> bool:
    vwap_ok = setup.get("structure_ok", False)
    ema_ok = ((setup.get("ema5m_side") == "below" and setup.get("side") == "SELL") or
              (setup.get("ema5m_side") == "above" and setup.get("side") == "BUY"))
    atr = max(setup.get("atr", 0.0) or 0.0, 1e-9)
    sw = setup.get("stop_width", 0.0) or 0.0
    vol_fit = (1.0 * atr <= sw <= 2.5 * atr)
    conf_ok = conf >= PRIORITY_CONF_THRESHOLD
    trend_ok = abs(regime_score) >= 1
    return vwap_ok and ema_ok and vol_fit and conf_ok and trend_ok

# ================== ALERTS ==================
def send_telegram(msg: str):
    if not TELEGRAM_TOKEN or not TELEGRAM_CHAT_ID:
        return
    try:
        r = requests.post(f"https://api.telegram.org/bot{TELEGRAM_TOKEN}/sendMessage",
                          json={"chat_id": TELEGRAM_CHAT_ID, "text": msg}, timeout=10)
        if DEBUG_COMPONENTS:
            print(f"[TG] {r.status_code}")
    except Exception as e:
        print(f"[WARN] Telegram send failed: {e}")

def post_discord(webhook: str, msg: str):
    if not webhook:
        return
    try:
        r = requests.post(webhook, json={"content": msg}, timeout=10)
        if r.status_code == 429:
            retry_after = r.headers.get("Retry-After")
            try:
                retry_s = int(float(retry_after))
            except Exception:
                retry_s = 1
            print(f"[WARN] Discord rate limited, sleeping {retry_s}s")
            time.sleep(retry_s)
            try:
                requests.post(webhook, json={"content": msg}, timeout=10)
            except Exception as e:
                print(f"[WARN] Discord retry failed: {e}")
                return
        if DEBUG_COMPONENTS:
            print(f"[DC] {r.status_code}")
    except Exception as e:
        print(f"[WARN] Discord send failed: {e}")

def send_discord_all(msg: str): post_discord(DISCORD_WEBHOOK_ALL, msg)
def send_discord_priority(msg: str): post_discord(DISCORD_WEBHOOK_PRIORITY, msg)

def action_hint(coin: str, side: str, setup: dict) -> str:
    e = setup["entry"]
    t1 = setup["tp1"]
    t2 = setup["tp2"]
    d0 = 1.0
    tp0 = e + d0 if side == "BUY" else e - d0
    return f"Hint: TP0 {tp0:.4f}, TP1 {t1:.4f}, TP2 {t2:.4f}"

def format_msg(coin: str, side: str, regime_score: float, setup: dict, conf: int) -> str:
    entry, stop, tp1, tp2 = setup["entry"], setup["stop"], setup["tp1"], setup["tp2"]
    sw = setup["stop_width"]
    atr1 = setup.get("atr") or 0.0
    atr_mult = (sw / max(atr1, 1e-9)) if atr1 else 0.0
    size = PER_TRADE_RISK_USD / max(sw, 1e-9)
    vwap_mark = "✓" if setup.get("structure_ok") else "×"
    ema_side = setup.get("ema5m_side", "na")
    ema_ok = ((ema_side == "above" and side == "BUY") or (ema_side == "below" and side == "SELL"))
    ema_mark = "✓" if ema_ok else "×"
    tier = "🚀" if is_priority(setup, regime_score, conf) else ("✅" if conf >= TELEGRAM_CONF_THRESHOLD else "🟨")
    hdr = f"{tier} {coin} {side} | Conf {conf}% | Trend {regime_score} | VWAP {vwap_mark} | 5m {ema_mark}"
    plan = f"Plan: E {entry:.4f} | SL {stop:.4f} | TP1 {tp1:.4f} | TP2 {tp2:.4f}"
    hint = action_hint(coin, side, setup)
    return f"[{('PRIORITY' if tier=='🚀' else 'REGULAR' if tier=='✅' else 'MANUAL')}] {hdr}\n{plan}\n{hint}"

# ================== ROUTING ==================
def adaptive_priority_threshold(coin: str) -> int:
    base = 68
    hist = list(CONF_HISTORY[coin])
    if len(hist) < 30:
        return base
    p80 = float(pd.Series(hist).quantile(0.8))
    return max(base, round(p80))

def route_alerts(coin: str, side: str, regime_score: int, setup: dict, conf: int, tag: str = ""):
    now = now_ts()
    ls = LAST_SENT[coin]
    fp = f"{coin}:{tag}:{setup.get('side')}:{regime_score}:{int(conf)}"
    last_activity_ts = max(ls["prio_ts"], ls["reg_ts"], ls["man_ts"])
    if fp == ls["fp"] and (now - last_activity_ts) < 180:
        if DEBUG_COMPONENTS:
            print(f"[DEDUP] skip duplicate {fp} for {coin}")
        return
    if tag == "PRIORITY":
        PRIO_BURST[coin].append(now)
        if len(PRIO_BURST[coin]) >= PRIO_BURST_COUNT and (now - PRIO_BURST[coin][0]) < PRIO_BURST_WINDOW_SEC:
            if conf < adaptive_priority_threshold(coin) + 3:
                if DEBUG_COMPONENTS:
                    print(f"[PRIO_BURST] suppress prio burst {coin}")
                return
        if now - ls["prio_ts"] < PRIO_MIN_GAP:
            if DEBUG_COMPONENTS:
                print(f"[PRIO_COOLDOWN] {coin} prio gap")
            return
    elif tag == "REGULAR":
        if now - ls["reg_ts"] < REG_MIN_GAP:
            if DEBUG_COMPONENTS:
                print(f"[REG_COOLDOWN] {coin} reg gap")
            return
    elif tag == "MANUAL":
        if now - ls["man_ts"] < MAN_MIN_GAP:
            if DEBUG_COMPONENTS:
                print(f"[MAN_COOLDOWN] {coin} man gap")
            return

    msg = format_msg(coin, side, regime_score, setup, conf)

    # Always send to general webhook for visibility
    send_discord_all(msg)

    if tag == "PRIORITY":
        ls["prio_ts"] = now
        ls["fp"] = fp
        role = f"<@&{DISCORD_PRIORITY_ROLE_ID}> " if DISCORD_PRIORITY_ROLE_ID else ""
        send_discord_priority(role + msg)
        if DEBUG_COMPONENTS:
            print(f"[PRIORITY] {coin} | Conf {conf}%")
        return

    if tag == "MANUAL":
        ls["man_ts"] = now
        ls["fp"] = fp
        if DEBUG_COMPONENTS:
            print(f"[MANUAL] {coin} | Conf {conf}%")
        return

    if tag == "REGULAR":
        ls["reg_ts"] = now
        ls["fp"] = fp
        if conf >= TELEGRAM_CONF_THRESHOLD:
            send_telegram(msg)
        if DEBUG_COMPONENTS:
            print(f"[REGULAR] {coin} | Conf {conf}%")
        return

# ================== ANALYZE (5m-driven) ==================
def in_overlap_hours(dt_utc: datetime) -> bool:
    h = dt_utc.hour
    return 12 <= h <= 17

def analyze_coin(coin: str):
    try:
        symbol = COINBASE_SYMBOLS.get(coin)
        if not symbol:
            print(f"[WARN] Unknown symbol for {coin}")
            return

        df1m = fetch_coinbase_candles(symbol, "1", 300)
        df5m = fetch_coinbase_candles(symbol, "5", 300)
        df1h = fetch_coinbase_candles(symbol, "60", 240)
        if df1m is None or df5m is None or df1h is None or df1m.empty or df5m.empty or df1h.empty:
            if DEBUG_COMPONENTS:
                print(f"[WARN] Skipping {coin}, missing data")
            return

        try:
            ts = int(df1m["time"].iloc[-1].timestamp())
            prev_ts = LAST_CANDLE_TS.get(symbol)
            live = LIVE_LAST.get(symbol)
            if prev_ts == ts:
                if live:
                    last_close = float(df1m["close"].iloc[-1])
                    if abs(live - last_close) / max(last_close, 1e-12) < LIVE_DEVIATION_BP:
                        if DEBUG_COMPONENTS:
                            print(f"[SKIP] {coin} unchanged candle and live within threshold")
                        return
                else:
                    if DEBUG_COMPONENTS:
                        print(f"[SKIP] {coin} unchanged candle and no live tick")
                    return
            LAST_CANDLE_TS[symbol] = ts
        except Exception:
            pass

        df1m = add_indicators(df1m)
        df5m = add_indicators(df5m)
        df1h = add_indicators(df1h)

        side5 = five_min_side(df5m)
        if side5 == "na":
            if DEBUG_COMPONENTS:
                print(f"[WARN] {coin} 5m EMAs not ready")
            return
        side = "BUY" if side5 == "above" else "SELL"

        entry = float(df1m["close"].iloc[-1])
        if pd.isna(entry):
            print(f"[WARN] {coin} last close NaN, skip")
            return

        vwap_val = compute_session_vwap(df1m)
        vwap_align = bool(vwap_val and ((entry > vwap_val and side == "BUY") or (entry < vwap_val and side == "SELL")))

        trigger_ok = one_min_trigger(df1m, side)

        ema5m_side, _ = ema_side_slope(df5m)
        regime = trend_score(df1h, df5m)

        # Stop handling: prefer swing; fallback/snap to ATR
        swing = detect_swing(df5m, side)
        atr1m = float(df1m["ATR14"].iloc[-1]) if "ATR14" in df1m and not df1m["ATR14"].isna().all() else None
        K = 1.8
        if swing is None or pd.isna(swing):
            if atr1m is None:
                if DEBUG_COMPONENTS:
                    print(f"[WARN] {coin} no ATR1m; skipping")
                return
            stop = entry - K * atr1m if side == "BUY" else entry + K * atr1m
        else:
            stop = float(swing)

        tp1 = entry + (entry - stop) if side == "BUY" else entry - (stop - entry)
        tp2 = entry + 2 * (entry - stop) if side == "BUY" else entry - 2 * (stop - entry)
        stop_width = abs(entry - stop)

        # ATR clamp with per-asset hi tweak and snapping
        hi_mult = 2.5
        if coin in ("ETH", "SOL"):
            hi_mult = 3.0
        if atr1m:
            lo = 1.0 * atr1m
            hi = hi_mult * atr1m
            if stop_width < lo:
                stop = entry - lo if side == "BUY" else entry + lo
                if DEBUG_COMPONENTS:
                    print(f"[SNAP] {coin} stop too tight -> moved to {stop:.6f} (lo={lo:.6f})")
            elif stop_width > hi:
                stop = entry - K * atr1m if side == "BUY" else entry + K * atr1m
                if DEBUG_COMPONENTS:
                    print(f"[SNAP] {coin} swing stop out-of-band -> snapped to ATR stop {stop:.6f} (hi={hi:.6f})")
            tp1 = entry + (entry - stop) if side == "BUY" else entry - (stop - entry)
            tp2 = entry + 2 * (entry - stop) if side == "BUY" else entry - 2 * (stop - entry)
            stop_width = abs(entry - stop)
            if not atr_clamp_ok(stop_width, atr1m, lo_mult=1.0, hi_mult=hi_mult):
                if DEBUG_COMPONENTS:
                    print(f"[SKIP] {coin} final stop {stop_width:.4f} out of ATR band [{lo:.4f}, {hi:.4f}] after snap")
                return

        setup = {
            "entry": entry, "stop": stop, "tp1": tp1, "tp2": tp2,
            "side": side, "stop_width": stop_width, "atr": atr1m,
            "ema5m_side": ema5m_side, "structure_ok": vwap_align
        }

        conf = compute_confidence(regime, ema5m_side, side, vwap_align, stop_width, atr1m)
        conf = int(min(100, max(0, conf + ema_slope_bonus_5m(df5m, side))))
        CONF_HISTORY[coin].append(conf)

        dyn_priority = adaptive_priority_threshold(coin)
        if in_overlap_hours(utcnow()):
            dyn_priority = max(0, dyn_priority - 1)

        promoted_to_priority = False
        last_prio = LAST_PRIORITY_TS[coin]
        minutes_since_prio = (now_ts() - last_prio) / 60 if last_prio else 999
        if conf >= dyn_priority:
            LAST_PRIORITY_TS[coin] = now_ts()
            DROUGHT_PROMOTED_COUNT[coin] = 0
        else:
            if minutes_since_prio >= 90 and vwap_align and ema5m_side and conf >= dyn_priority - 2:
                promoted_to_priority = True
                LAST_PRIORITY_TS[coin] = now_ts()
                DROUGHT_PROMOTED_COUNT[coin] += 1

        tag = "PRIORITY" if (promoted_to_priority or conf >= dyn_priority) else ("REGULAR" if conf >= TELEGRAM_CONF_THRESHOLD else "MANUAL")

        # require higher timeframe align for priority unless very strong
        ema15_align = False
        if "EMA20" in df1h and "EMA50" in df1h:
            ema15_align = df1h["EMA20"].iloc[-1] > df1h["EMA50"].iloc[-1] if side == "BUY" else df1h["EMA20"].iloc[-1] < df1h["EMA50"].iloc[-1]
        if tag == "PRIORITY" and not ema15_align and conf < dyn_priority + 5:
            tag = "REGULAR"
            if DEBUG_COMPONENTS:
                print(f"[PRIO_GATE] demoted {coin} to REGULAR (no 1h align)")

        route_alerts(coin, side, regime, setup, conf, tag=tag)

        ts = datetime.now(timezone.utc).isoformat()
        log_sheet([ts, coin, side, entry, stop, tp1, tp2, conf, regime, ema5m_side, atr1m or "", stop_width, tag])

        if DEBUG_COMPONENTS:
            print(f"[DBG] {coin} side={side} conf={conf} dynPrio={dyn_priority} tag={tag}")

    except Exception as e:
        print(f"[ERROR] analyze_coin {coin}: {e}")
        traceback.print_exc()

# ================== WEBSOCKET (optional) ==================
def ws_worker(symbols):
    url = "wss://ws-feed.exchange.coinbase.com"
    subs = {"type": "subscribe", "channels": [{"name": "ticker", "product_ids": symbols}]}
    while True:
        try:
            ws = websocket.create_connection(url, timeout=10)
            ws.send(json.dumps(subs))
            while True:
                raw = ws.recv()
                if not raw:
                    time.sleep(0.1)
                    continue
                msg = json.loads(raw)
                if msg.get("type") == "ticker" and "product_id" in msg and "price" in msg:
                    LIVE_LAST[msg["product_id"]] = float(msg["price"])
        except Exception:
            time.sleep(3)

# ================== MAIN LOOP ==================
def main_loop():
    init_sheet()
    if WEBSOCKET_AVAILABLE:
        symbols = [COINBASE_SYMBOLS[c] for c in MARKETS if c in COINBASE_SYMBOLS]
        threading.Thread(target=ws_worker, args=(symbols,), daemon=True).start()
    else:
        print("[WARN] websocket-client not available; running without live tick updates")
    tick = 0
    while True:
        for coin in MARKETS:
            analyze_coin(coin)
        tick += 1
        if tick % (3600 // max(1, POLL_INTERVAL)) == 0:
            print(f"[HEARTBEAT] {utcnow().isoformat()} running…")
        time.sleep(POLL_INTERVAL)

if __name__ == "__main__":
    main_loop()
# ...existing code...

Streaming output truncated to the last 5000 lines.
[INFO] Fetching SOL-USD 1m candles...
[INFO] Fetching SOL-USD 5m candles...
[INFO] Fetching SOL-USD 60m candles...
[SKIP] SOL unchanged candle and live within threshold
[INFO] Fetching XRP-USD 1m candles...
[INFO] Fetching XRP-USD 5m candles...
[INFO] Fetching XRP-USD 60m candles...
[SKIP] XRP unchanged candle and live within threshold
[INFO] Fetching AVAX-USD 1m candles...
[INFO] Fetching AVAX-USD 5m candles...
[INFO] Fetching AVAX-USD 60m candles...
[SKIP] AVAX unchanged candle and live within threshold
[INFO] Fetching BTC-USD 1m candles...
[INFO] Fetching BTC-USD 5m candles...
[INFO] Fetching BTC-USD 60m candles...
[SNAP] BTC swing stop out-of-band -> snapped to ATR stop 122094.693714 (hi=93.064286)
[DC] 204
[MANUAL] BTC | Conf 63%
[DBG] BTC side=BUY conf=63 dynPrio=78 tag=MANUAL
[INFO] Fetching ETH-USD 1m candles...
[INFO] Fetching ETH-USD 5m candles...
[INFO] Fetching ETH-USD 60m candles...
[SNAP] ETH swing stop out-of-band -> sna